In [1]:
# Demat Account Details

USER_NAME = 'P60493342'
PWD = '2536'
API_KEY = 'PKuyM2GK'
TOTP = "LFXFA6K3B4JYOO4RFPH7WDB4Q4"

CORRELATION_ID = "63cfcac0-5c59-49ad-9f50-f3d1e28ff118"
TOKEN_MAP = None
FEED_TOKEN = None
SMART_API_OBJ = None

# Short name of share & Exchange & Number

exchange_segment = 'NSE'
symbol = 'NIFTYBEES-EQ'
symbol_token = None   # 14366

fund_percentage = 50  # %

In [3]:
# For Connect Demat Account

from SmartApi import SmartConnect
import pyotp
from logzero import logger
import pandas as pd
import numpy as np
import pytz
import requests
from talib.abstract import *
from datetime import *
import requests

In [17]:
obj = SmartConnect(API_KEY)

try:
    totp = pyotp.TOTP(TOTP).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

login = obj.generateSession(USER_NAME,PWD,totp)

if login['status'] == False:
    logger.error(login)
    
else:
    # login api call
   
    authToken = login['data']['jwtToken']
    
    refreshToken = login['data']['refreshToken']
    obj.generateToken(refreshToken)
    
    # fetch the feedtoken
    feedToken = obj.getfeedToken()
    
    # fetch User Profile
    res = obj.getProfile(refreshToken)
    My_Info = pd.DataFrame.from_dict(res)

symbol_token = obj.searchScrip(exchange_segment,symbol)['data'][0]['symboltoken']
fund_to_be_used = ( float(obj.rmsLimit()['data']['availablecash']) * fund_percentage ) / 100

display(My_Info["data"])
print(f"Fund To be Used : {fund_to_be_used}")

[I 241223 13:44:56 smartConnect:124] in pool
[I 241223 13:44:59 smartConnect:496] Search successful. Found 1 trading symbols for the given query:
    1. exchange: NSE, tradingsymbol: NIFTYBEES-EQ, symboltoken: 10576


clientcode                                               P60493342
name                                             PATHAN FAIJANKHAN
email                                                             
mobileno                                                          
exchanges        [nse_fo, nse_cm, cde_fo, ncx_fo, bse_fo, bse_c...
products                          [MARGIN, MIS, NRML, CNC, CO, BO]
lastlogintime                                                     
broker                                                            
Name: data, dtype: object

Fund To be Used : 490.79


In [15]:
# For Symbol Data Download  

def intializeSymbolTokenMap():
    url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
    d = requests.get(url).json()
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry'], format='%d%b%Y')
    token_df = token_df.astype({'strike': float})
    global TOKEN_MAP 
    TOKEN_MAP = token_df

intializeSymbolTokenMap()
TOKEN_MAP

,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size
0,99926000,Nifty 50,NIFTY,NaT,0.000000e+00,1,AMXIDX,NSE,0.000000
1,99926001,Nifty GrowSect 15,NIFTY GROWSECT 15,NaT,0.000000e+00,1,AMXIDX,NSE,0.000000
2,99926002,Nifty50 PR 2x Lev,NIFTY50 PR 2X LEV,NaT,0.000000e+00,1,AMXIDX,NSE,0.000000
3,99926004,Nifty 500,NIFTY 500,NaT,0.000000e+00,1,AMXIDX,NSE,0.000000
4,99926008,Nifty IT,NIFTY IT,NaT,0.000000e+00,1,AMXIDX,NSE,0.000000
...,...,...,...,...,...,...,...,...,...
120008,127689,SILVERM25FEB103000CE,SILVERM,2025-02-21,1.030000e+09,1,OPTFUT,NCO,5000.000000
120009,127719,SILVERM25APR101000CE,SILVERM,2025-04-23,1.010000e+09,1,OPTFUT,NCO,5000.000000
120010,127727,SILVERM25APR102000CE,SILVERM,2025-04-23,1.020000e+09,1,OPTFUT,NCO,5000.000000
120011,127751,SILVERM25APR105000CE,SILVERM,2025-04-23,1.050000e+09,1,OPTFUT,NCO,5000.000000


In [5]:
# # For Directly Symbol Token

# def get_symbol_token(exchange,name_symbol) :
#     try : 
#         token = obj.searchScrip(exchange,name_symbol)['data'][0]['symboltoken']
#         return token
#     except Exception as e:
#         logger.exception(f"Symbol Token Not Found : {e}")

# # exchange = "NSE"
# # name_symbol = "SBIN-EQ"
# # t = get_symbol_token(exchange,name_symbol)
# # t

In [6]:
# # For Option & Future Information

# def getTokenInfo (exch_seg, instrumenttype,symbol,strike_price,pe_ce):
    
#     df = TOKEN_MAP
#     strike_price = strike_price*100
    
#     if exch_seg == 'NSE':
#         eq_df = df[(df['exch_seg'] == 'NSE') & (df['symbol'].str.contains('EQ')) ]
#         return eq_df[eq_df['name'] == symbol]
        
#     elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
#         return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    
#     elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
#         return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])

# tokenInfo = getTokenInfo('NFO','OPTIDX','NIFTY',25000,'CE').iloc[0]
# print(tokenInfo)

# symbol  = tokenInfo['symbol']
# token = tokenInfo['token']
# lot = int(tokenInfo['lotsize'])

# print(symbol, token, lot)

In [19]:
# For Historic Data

def historical_data(exchange_segment,symbol_token,time_frame,from_date,to_date) :

    try:
        historicParam={
        "exchange": exchange_segment,      #  NSE
        "symboltoken": symbol_token ,      #  3045
        "interval": time_frame ,           #  ONE_MINUTE
        "fromdate": from_date ,            #  2024-11-20 09:15
        "todate": to_date                  #  2024-11-30 15:30
        }
        
        dt = obj.getCandleData(historicParam)['data']
        candledata = pd.DataFrame(dt,columns = ["Date & Time","Open","High","Low","Close","Volume"] )

        # Convert Time column to datetime
        candledata['Date & Time'] = pd.to_datetime(candledata['Date & Time'])

        # date & time Format
        candledata['Date & Time'] = candledata['Date & Time'].dt.strftime('%d-%m-%Y  %H:%M')
    
        # Set Date & Time As Index
        # candledata = candledata.set_index('Date & Time')
    
        return candledata
    
    except Exception as e:
        logger.exception(f"Historic Api failed: {e}")

# Historic Data 

symbol_token = symbol_token
time_frame = "FIFTEEN_MINUTE"
from_date = datetime.now() - timedelta(days=200)
from_date = from_date.strftime("%Y-%m-%d %H:%M")
to_date = datetime.now().strftime("%Y-%m-%d %H:%M")

# Get historical data
data = historical_data(exchange_segment,symbol_token,time_frame,from_date,to_date)
data.iloc[-50:]

,Date & Time,Open,High,Low,Close,Volume
3329,19-12-2024 14:00,267.98,268.17,267.91,268.07,93667
3330,19-12-2024 14:15,268.09,268.15,267.83,267.96,158374
3331,19-12-2024 14:30,267.94,268.09,267.76,267.94,260095
3332,19-12-2024 14:45,267.94,268.00,267.90,267.93,136317
3333,19-12-2024 15:00,267.93,268.13,267.87,268.07,324014
3334,19-12-2024 15:15,268.08,276.42,267.80,268.50,602505
3335,20-12-2024 09:15,267.11,268.99,267.11,267.65,396170
3336,20-12-2024 09:30,267.64,269.32,267.50,269.09,285338
3337,20-12-2024 09:45,269.09,269.09,266.15,266.88,737595
3338,20-12-2024 10:00,266.84,266.98,266.40,266.89,303514


In [21]:
# MACD Implementation

MACD_data = pd.DataFrame()

MACD_data['MACD'], MACD_data['MACD_Signal'], MACD_data['MACD_Hist'] = MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9) 
MACD_data['MACD_Buy'] = (MACD_data['MACD'] > MACD_data['MACD_Signal']) 
MACD_data['MACD_Sell'] = (MACD_data['MACD'] < MACD_data['MACD_Signal'])

MACD_data[-50:]

,MACD,MACD_Signal,MACD_Hist,MACD_Buy,MACD_Sell
3329,-0.569592,-0.684361,0.114769,True,False
3330,-0.542676,-0.656024,0.113348,True,False
3331,-0.516999,-0.628219,0.111220,True,False
3332,-0.491787,-0.600932,0.109145,True,False
3333,-0.455263,-0.571798,0.116536,True,False
3334,-0.387156,-0.534870,0.147714,True,False
3335,-0.397190,-0.507334,0.110144,True,False
3336,-0.285654,-0.462998,0.177344,True,False
3337,-0.371309,-0.444660,0.073351,True,False
3338,-0.433388,-0.442406,0.009018,True,False


In [23]:
# Moving Average Crossover Implementation

MAC_data = pd.DataFrame()

MAC_data['SMA50'] = data['Close'].rolling(window=50).mean() 
MAC_data['SMA200'] = data['Close'].rolling(window=200).mean() 
MAC_data['MAC_Buy'] = (MAC_data['SMA50'] > MAC_data['SMA200']) 
MAC_data['MAC_Sell'] = (MAC_data['SMA50'] < MAC_data['SMA200'])

MAC_data[-50:]

,SMA50,SMA200,MAC_Buy,MAC_Sell
3329,269.9400,273.41870,False,True
3330,269.8614,273.38285,False,True
3331,269.7768,273.34705,False,True
3332,269.6914,273.31090,False,True
3333,269.6072,273.27550,False,True
3334,269.5342,273.24205,False,True
3335,269.4518,273.20575,False,True
3336,269.3800,273.17370,False,True
3337,269.2882,273.12965,False,True
3338,269.2020,273.08560,False,True


In [25]:
# RSI Implementation

# # Calculate daily price changes
# delta = data['Close'].diff()

# # Separate gains and losses
# gain = np.where(delta > 0, delta, 0)
# loss = np.where(delta < 0, -delta, 0)

# # Calculate average gain and loss using rolling mean
# avg_gain = pd.Series(gain).rolling(window=rsi_period, min_periods=1).mean()
# avg_loss = pd.Series(loss).rolling(window=rsi_period, min_periods=1).mean()

# # Calculate RS and RSI
# rs = avg_gain / avg_loss
# rsi = 100 - (100 / (1 + rs))

# # Add RSI column to the DataFrame
# data['RSI'] = rsi

# RSI parameters
rsi_period = 14

RSI_data = pd.DataFrame()

RSI_data['RSI'] = RSI(data.Close, rsi_period)
RSI_data['RSI_Buy'] = (RSI_data['RSI'] < 30) 
RSI_data['RSI_Sell'] = (RSI_data['RSI'] > 70)
RSI_data[-50:]

,RSI,RSI_Buy,RSI_Sell
3329,37.792572,False,False
3330,36.391707,False,False
3331,36.129483,False,False
3332,35.989844,False,False
3333,39.514432,False,False
3334,48.833459,False,False
3335,36.772599,False,False
3336,56.412865,False,False
3337,37.275563,False,False
3338,37.379080,False,False


In [27]:
# SuperTrend Implementation

atr_period=20
multiplier=3

supertrend_data = pd.DataFrame(index=data.index)
hl2 = (data['High'] + data['Low']) / 2
supertrend_data['ATR'] = ATR(data['High'], data['Low'], data['Close'], timeperiod=atr_period)
supertrend_data['Upper Band'] = hl2 + (multiplier * supertrend_data['ATR'])
supertrend_data['Lower Band'] = hl2 - (multiplier * supertrend_data['ATR'])
supertrend_data['In Uptrend'] = True

for current in range(1, len(supertrend_data.index)):
    previous = current - 1

    if data['Close'].iloc[current] > supertrend_data['Upper Band'].iloc[previous]:
        supertrend_data.loc[supertrend_data.index[current], 'In Uptrend'] = True
    elif data['Close'].iloc[current] < supertrend_data['Lower Band'].iloc[previous]:
        supertrend_data.loc[supertrend_data.index[current], 'In Uptrend'] = False
    else:
        supertrend_data.loc[supertrend_data.index[current], 'In Uptrend'] = supertrend_data.loc[supertrend_data.index[previous], 'In Uptrend']
        if supertrend_data.loc[supertrend_data.index[current], 'In Uptrend'] and supertrend_data['Lower Band'].iloc[current] < supertrend_data['Lower Band'].iloc[previous]:
            supertrend_data.loc[supertrend_data.index[current], 'Lower Band'] = supertrend_data['Lower Band'].iloc[previous]
        if not supertrend_data.loc[supertrend_data.index[current], 'In Uptrend'] and supertrend_data['Upper Band'].iloc[current] > supertrend_data['Upper Band'].iloc[previous]:
            supertrend_data.loc[supertrend_data.index[current], 'Upper Band'] = supertrend_data['Upper Band'].iloc[previous]

supertrend_data['SuperTrend_Buy'] = (supertrend_data['In Uptrend'])
supertrend_data['SuperTrend_Sell'] = (~supertrend_data['In Uptrend'])

supertrend_data

,ATR,Upper Band,Lower Band,In Uptrend,SuperTrend_Buy,SuperTrend_Sell
0,NaN,NaN,NaN,True,True,False
1,NaN,NaN,NaN,True,True,False
2,NaN,NaN,NaN,True,True,False
3,NaN,NaN,NaN,True,True,False
4,NaN,NaN,NaN,True,True,False
...,...,...,...,...,...,...
3374,0.827770,267.798309,264.083652,True,True,False
3375,0.815381,267.596144,264.083652,True,True,False
3376,0.798112,267.309337,264.083652,True,True,False
3377,0.788207,267.214620,264.083652,True,True,False


In [29]:
# Create DataFrame for Combined_Buy with all individual buy signals 
combined_buy = pd.DataFrame()

combined_buy['MACD_Buy'] = MACD_data['MACD_Buy']
combined_buy['MAC_Buy'] = MAC_data['MAC_Buy']
combined_buy['RSI_Buy'] = RSI_data['RSI_Buy']
combined_buy['SuperTrend_Buy'] = supertrend_data['SuperTrend_Buy']
combined_buy['Total Buy'] = combined_buy[['MACD_Buy','MAC_Buy','RSI_Buy','SuperTrend_Buy']].sum(axis=1)

combined_buy.iloc[-50:]

,MACD_Buy,MAC_Buy,RSI_Buy,SuperTrend_Buy,Total Buy
3329,True,False,False,False,1
3330,True,False,False,False,1
3331,True,False,False,False,1
3332,True,False,False,False,1
3333,True,False,False,False,1
3334,True,False,False,False,1
3335,True,False,False,False,1
3336,True,False,False,False,1
3337,True,False,False,False,1
3338,True,False,False,False,1


In [31]:
# Create DataFrame for Combined_Sell with all individual Sell signals 
combined_sell = pd.DataFrame()

combined_sell['MACD_Sell'] = MACD_data['MACD_Sell']
combined_sell['MAC_Sell'] = MAC_data['MAC_Sell']
combined_sell['RSI_Sell'] = RSI_data['RSI_Sell']
combined_sell['SuperTrend_Sell'] = supertrend_data['SuperTrend_Sell']
combined_sell['Total Sell'] = combined_sell[['MACD_Sell','MAC_Sell','RSI_Sell','SuperTrend_Sell']].sum(axis=1)

combined_sell.iloc[-50:]

,MACD_Sell,MAC_Sell,RSI_Sell,SuperTrend_Sell,Total Sell
3329,False,True,False,True,2
3330,False,True,False,True,2
3331,False,True,False,True,2
3332,False,True,False,True,2
3333,False,True,False,True,2
3334,False,True,False,True,2
3335,False,True,False,True,2
3336,False,True,False,True,2
3337,False,True,False,True,2
3338,False,True,False,True,2


In [37]:
data['Total Buy'] = combined_buy['Total Buy'] 
data['Total Sell'] = combined_sell['Total Sell']

# Use np.where to create the 'Action' column based on the conditions 
data['Action'] = np.where(data['Total Buy'] >= 1, 'Buy', np.where(data['Total Sell'] >= 3, 'Sell', '-'))

# Convert to excel sheet
data.to_excel(r'C:\Users\faija\OneDrive\Desktop\Study\Algo Trading Project\Past Data\data.xlsx',index=False)

# Data Manupalation For Testing
# data['Action'].iloc[-1] = 'Buy'

data[-50:]

,Date & Time,Open,High,Low,Close,Volume,Total Buy,Total Sell,Action
3329,19-12-2024 14:00,267.98,268.17,267.91,268.07,93667,1,2,Buy
3330,19-12-2024 14:15,268.09,268.15,267.83,267.96,158374,1,2,Buy
3331,19-12-2024 14:30,267.94,268.09,267.76,267.94,260095,1,2,Buy
3332,19-12-2024 14:45,267.94,268.00,267.90,267.93,136317,1,2,Buy
3333,19-12-2024 15:00,267.93,268.13,267.87,268.07,324014,1,2,Buy
3334,19-12-2024 15:15,268.08,276.42,267.80,268.50,602505,1,2,Buy
3335,20-12-2024 09:15,267.11,268.99,267.11,267.65,396170,1,2,Buy
3336,20-12-2024 09:30,267.64,269.32,267.50,269.09,285338,1,2,Buy
3337,20-12-2024 09:45,269.09,269.09,266.15,266.88,737595,1,2,Buy
3338,20-12-2024 10:00,266.84,266.98,266.40,266.89,303514,1,2,Buy


In [39]:
# Send A Message On Telegram

def telegram_bot_sendtext(bot_message):
    bot_token = '7988064526:AAFNx1tSItQXE9BobdTf0ijZbe-v6BWnjqc' 
    bot_chatID = '6066882633'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + \
                '&parse_mode=HTML&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

In [41]:
# For Placing Order

def place_order(symbol,symbol_token,buy_sell,exchange_segment,price,quantity) :
    
    try :
        orderparams = {
            "variety": "NORMAL",               
            "tradingsymbol": symbol ,         #  SBIN-EQ
            "symboltoken": symbol_token ,     #  3045
            "transactiontype": buy_sell ,     #  BUY,SELL
            "exchange": exchange_segment ,    #  NSE,BSE,NFO,MCX,BFO,CDS
            "ordertype": "MARKET" ,           #  MARKET,LIMIT
            "producttype": "INTRADAY" ,       #  DELIVERY,CARRYFORWARD
            "duration": "DAY",                
            "price": 0 ,                  
            "squareoff": 30.00 ,          
            "stoploss": 15.00 ,            
            "quantity": quantity              
            }
        
        # Place an order and return the order ID
        order_id = obj.placeOrder(orderparams)
        return order_id
        
    except Exception as e:
        logger.exception(f"Order placement failed : {e}")

# Place Order

if data['Action'].iloc[-1] == 'Buy' :
    buy_sell = "BUY"
elif data['Action'].iloc[-1] == 'Sell' :
    buy_sell = "SELL"
else :
    buy_sell = None
    print("No Need To Trade")
    test = telegram_bot_sendtext("No Need To Trade")
    print(test)

if buy_sell != None : 
    
    price = obj.ltpData(exchange_segment,symbol,symbol_token)['data']['ltp']

    if fund_to_be_used >= price :
    
        quantity = round(fund_to_be_used/price)
        
        try : 
    
            order_id = place_order(symbol,symbol_token,buy_sell,exchange_segment,price,quantity)
            print("Order Placed Succesfully")
            print(f"Buy/Sell : {buy_sell}")
            print(f"Name : {symbol}")
            print(f"Quantity : {quantity}")
            print(f"Order ID : {order_id}")

            print(obj.position()['data'])
    
            # Telegram Message Send
            msg = symbol + "  " + buy_sell
            test = telegram_bot_sendtext(msg)
            print(test)
    
        except Exception as e:
            logger.exception("Problem") #(f"Order placement failed : {e}")

    else : 
        print("Not Enough Available Fund")
        test = telegram_bot_sendtext("Not Enough Available Fund")
        print(test)

Order Placed Succesfully
Buy/Sell : BUY
Name : NIFTYBEES-EQ
Quantity : 2
Order ID : 241223000679482
[{'symboltoken': '10576', 'symbolname': 'NIFTYBEES', 'instrumenttype': '', 'priceden': '1.00', 'pricenum': '1.00', 'genden': '1.00', 'gennum': '1.00', 'precision': '2', 'multiplier': '-1', 'boardlotsize': '1', 'exchange': 'NSE', 'producttype': 'INTRADAY', 'tradingsymbol': 'NIFTYBEES-EQ', 'symbolgroup': 'EQ', 'strikeprice': '-1.0', 'optiontype': '', 'expirydate': '', 'lotsize': '1', 'cfbuyqty': '0', 'cfsellqty': '0', 'cfbuyamount': '0.00', 'cfsellamount': '0.00', 'buyavgprice': '264.82', 'sellavgprice': '264.65', 'avgnetprice': '265.00', 'netvalue': '-530.00', 'netqty': '2', 'totalbuyvalue': '1059.30', 'totalsellvalue': '529.30', 'cfbuyavgprice': '0.00', 'cfsellavgprice': '0.00', 'totalbuyavgprice': '264.82', 'totalsellavgprice': '264.65', 'netprice': '265.00', 'buyqty': '4', 'sellqty': '2', 'buyamount': '1059.30', 'sellamount': '529.30', 'pnl': '0.06', 'realised': '-0.34', 'unrealised': 

In [43]:
position = obj.position()['data']
position

[{'symboltoken': '10576',
  'symbolname': 'NIFTYBEES',
  'instrumenttype': '',
  'priceden': '1.00',
  'pricenum': '1.00',
  'genden': '1.00',
  'gennum': '1.00',
  'precision': '2',
  'multiplier': '-1',
  'boardlotsize': '1',
  'exchange': 'NSE',
  'producttype': 'INTRADAY',
  'tradingsymbol': 'NIFTYBEES-EQ',
  'symbolgroup': 'EQ',
  'strikeprice': '-1.0',
  'optiontype': '',
  'expirydate': '',
  'lotsize': '1',
  'cfbuyqty': '0',
  'cfsellqty': '0',
  'cfbuyamount': '0.00',
  'cfsellamount': '0.00',
  'buyavgprice': '264.82',
  'sellavgprice': '264.78',
  'avgnetprice': '0.00',
  'netvalue': '-0.18',
  'netqty': '0',
  'totalbuyvalue': '1059.30',
  'totalsellvalue': '1059.12',
  'cfbuyavgprice': '0.00',
  'cfsellavgprice': '0.00',
  'totalbuyavgprice': '264.82',
  'totalsellavgprice': '264.78',
  'netprice': '0.00',
  'buyqty': '4',
  'sellqty': '4',
  'buyamount': '1059.30',
  'sellamount': '1059.12',
  'pnl': '-0.16',
  'realised': '-0.16',
  'unrealised': '-0.00',
  'ltp': '264.

In [45]:
# Portfolio

try :
    f = obj.holding()['data']
    fund = pd.DataFrame(f)
    fund = fund.drop(['t1quantity','realisedquantity','authorisedquantity','collateralquantity','collateraltype','haircut'], axis=1)
    print(fund)
except Exception as e:
    logger.exception(f"Holdings Fetching failed: {e}")

  tradingsymbol exchange          isin  quantity   product  averageprice  \
0     AFCONS-EQ      NSE  INE101I01011        32  DELIVERY        463.00   
1       IDEA-EQ      NSE  INE669E01016         1  DELIVERY          8.42   
2    CPSEETF-EQ      NSE  INF457M01133       150  DELIVERY        101.41   
3    USHAFIN-SM      NSE  INE0LS001014       800  DELIVERY        168.00   
4    HYUNDAI-EQ      NSE  INE0V6F01027         7  DELIVERY       1960.00   

       ltp symboltoken    close  profitandloss  pnlpercentage  
0   525.20       25977   520.25         1990.0          13.43  
1     7.43       14366     7.40           -1.0         -11.71  
2    85.55        2328    85.81        -2379.0         -15.64  
3   120.00       25952   118.70       -38400.0         -28.57  
4  1755.75       25844  1766.20        -1430.0         -10.42  


In [47]:
# Available Fund

fund = obj.rmsLimit()['data']['availablecash']
fund

'941.4000'

In [19]:
# logout

try:
    logout = obj.terminateSession(USER_NAME)
    print("Logout Successfull...")
    
except Exception as e:
    logger.exception(f"Logout failed: {e}")

Logout Successfull...
